In [12]:
import pandas as pd
import requests
import schedule
import time
import os

In [15]:
diretorio = r'C:\rafa\bd_dolar\banco_dolar_python'  # Use um caminho absoluto
arquivo_parquet = os.path.join(diretorio, 'dolar_historico.parquet')

In [20]:
# Função para obter a cotação do dólar
def obter_cotacao_dolar():
    # Substitua pela sua chave de API, se necessário
    url = 'https://api.exchangerate-api.com/v4/latest/USD'
    response = requests.get(url)
    data = response.json()
    
    # Obtenha a cotação do dólar em relação ao real
    cotacao_dolar = data['rates']['BRL']
    return cotacao_dolar

In [19]:
# Função para atualizar o arquivo Parquet
def atualizar_historico():
    # Obtenha a cotação do dólar
    cotacao = obter_cotacao_dolar()
    data_atual = pd.Timestamp.now()
    

In [21]:
# Função para atualizar o arquivo Parquet
def atualizar_historico():
    # Obtenha a cotação do dólar
    cotacao = obter_cotacao_dolar()
    data_atual = pd.Timestamp.now()

    # Crie um DataFrame com os dados
    novo_dado = pd.DataFrame({
        'data': [data_atual],
        'dolar': [cotacao]
    })

    # Leia o arquivo existente, se ele existir
    try:
        df_existente = pd.read_parquet(arquivo_parquet)
        # Concatene os novos dados com os existentes
        df_atualizado = pd.concat([df_existente, novo_dado], ignore_index=True)
    except FileNotFoundError:
        # Se o arquivo não existir, use o novo dado como o DataFrame
        df_atualizado = novo_dado

    # Salve o DataFrame atualizado no arquivo Parquet
    df_atualizado.to_parquet(arquivo_parquet, index=False)

In [25]:
# Agende a atualização a cada 30 minutos
schedule.every(30).minutes.do(atualizar_historico)
    
# Loop para manter o agendamento
def run_schedule():
    while True:
        schedule.run_pending()
        time.sleep(1)